In [12]:
from enum import auto
import pandas as pd
import numpy as np
import urllib
import sqlalchemy as sa
import warnings
from unidecode import unidecode
import os 
warnings.filterwarnings("ignore")

C:\Users\bustossanchez.9\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [13]:
def remove_accents(x):

    #convert plain text to utf-8
    u = unidecode(x, "utf-8")
    #convert utf-8 to normal text
    return (unidecode(u))

In [14]:
def HC_cleaning():
    file_path= 'HC/'+os.listdir('HC')[0]
    HC=pd.read_csv(file_path,  delimiter=';', encoding= 'unicode_escape')
    HC=HC.iloc[:,[0,2,6,8,10,14,18,26,28]]
    HC.columns=HC.columns.str.lower()
    
    HC.rename(columns={"agent_name":"agentname",
                        "vlookup":"agentid",
                        "tenure inactive (since ops)":"tenure"
                                              }, inplace=True)
        
    HC=HC.replace(",","0", regex=True)
    HC=HC.replace("-","0", regex=True)
    HC=HC.apply(lambda x: x.astype(str).str.lower())

    HC["agentname"]=HC["agentname"].apply(lambda x: remove_accents(x))
    HC['agentid']=HC["agentid"].astype('int64')
    return HC

In [15]:
def read_surveys():
    
    file_path= 'input/'+os.listdir('input')[0]
    RawData=pd.read_excel( file_path , header=4)

    #Column names to lowercase, drop unecessary fields and replace name
    RawData.columns=RawData.columns.str.lower()
    RawData=RawData.drop(RawData.columns[[1,2,6]], axis=1)
    RawData=RawData.drop(['contact link'], axis=1)
    RawData.rename(columns={"login_id":"agentid",
                                },
                                inplace=True
        )

    RawData.columns=[name.replace("categories.","").replace("`","") for name in RawData.columns]

    HC=HC_cleaning()
    #merge and create new table
    RawUpdated=pd.merge(HC, RawData, on="agentid", how="right")

    #Replacing hit and miss with 1's and 0's and leaving the columns as factors
    #RawUpdated.iloc[:,15:]=RawUpdated.iloc[:,15:].replace("Hit",1, regex=True).replace("Miss",0, regex=True)
    RawUpdated=RawUpdated.replace("Hit",1, regex=True).replace("Miss",0, regex=True)
    #Rename columns by component name
    #RawUpdated.columns=RawUpdated.columns[:15].to_list()+[name.split(".")[2] if len(name.split("."))==3 else name.replace("."," ") for name in RawUpdated.columns[15:]]
    RawUpdated.columns=[name.split(".")[2] if len(name.split("."))==3 else name.replace("."," ") for name in RawUpdated.columns]


    #Drop duplicates by EurekaID and columns duplicated
    RawUpdated.drop_duplicates(subset=["eureka id"], inplace=True)

    RawUpdated=RawUpdated[(RawUpdated["outliers excessive silence"]==0) 
                    &(RawUpdated["outliers longest contacts"]==0)
                    &(RawUpdated["agentid"]!=0)
                    &(RawUpdated["outliers spanish calls"]==0)]

    #Sort and reset_index
    RawUpdated=RawUpdated.sort_values(by="eureka id").reset_index(drop=True)                

    return RawUpdated

In [16]:
dict_categories_agent_cps= {

    'opening_cps' : {   
        'negative':[    'the agent dont uses effective listening skills',  ## Negative??          
                        'agent does not acknowledge the customer request',  ##negative??? 
                        'agent does not conveys urgency to assist',
                        'unnecessary information'],  ## Negative??

        'positive':[    'agent uses effective listening skills',
                        'agent demonstrates willingness to assist',
                        'thank you for calling']
                    },
    'investigation_cps' : {

                'damage from service center' : ['ask if customer has contacted the sc for damage'],

                'file a claim' : [  'adh question',
                                    'commerical vs personal',
                                    'how was the unit being used',
                                    'model number',
                                    'the date when the problem occurred',
                                    'time frame to call the customer',
                                    'what led up to issue'],

                'food loss claim':[ 'itemized list of food loss',
                                    'the agent tells the customer to send the info',
                                    'the failure is a no cooling or no power'],

                'near store question':['near store'],

                'offer status':[    'check has been approved',
                                    'gift card has been approved',
                                    'replacement',
                                    'store credit has been approved'],

                'personalized interaction' : [  'agent asks the required information',
                                                'callers name'],

                'registration' : [  'how much pay for the warranty',
                                    'product registration',
                                    'what is the model number',
                                    'what is the name of the product',
                                    'what is the purchase date']
                                    },
   
   'resolution_cps' : {

                'damage from sc' : [ 'you need to contact the sc'],

                'dead air time' : { 
                    
                    'positive': [     'agent did met hold time',
                                        'agent thanks for waiting'],

                    'negative' :[  'agent did not met dead air time']
                                    },

                'res_food loss claim' : ['recite script'],

                'sc customer will be contacted' : ['customer will be contacted'],

                'service center appointment' : ['status of the appointment or missed']
                        },

    'closing_cps' : {
                'closing call cps' : [      'proper closing'],
                
                'transfering cat cps' : [   'transfer',
                                            'transfer to supervisor']
                    },

    'legal_cps' : [ #'attorney',
                    'confidential information',
                    'internal procedure',
                    'social network'],

    'amynta_experiences' : {
        'negative':     ['blames the customer',
                            'colloquial words',
                            'communicates with respect'],
        'positive':[    'empathy and interest']         
                            }
}

TCD_cps = ['tcd tpc acccount updates',
            'tcd tpc cancelation request',
            'tcd tpc claim status  offer status',
            'tcd tpc claim status appointment status',
            'tcd tpc claim status parts status',
            'tcd tpc claim status service center', 
            'tcd tpc file a claim',
            'tcd tpc general inquiry', 
            'tcd tpc registration']


dict_categories_brand_cps= {
    'contract term and conditions':    [ 'contract expired',
                                        'contract fulfilled',
                                        'contract incomplete',
                                        'did not review terms and conditions',
                                        'no contract found',
                                        'no cover issue'],

    'service center issues':            [ 'the customer cant reach the sc',
                                        'the customer perceives unsatisfactory service',
                                        'there is no timely response from sc']
                            }

channel_categories = [ 'been on hold for so long time on the queue',
                        'been on hold or waiting',
                        'i dont want to wait',
                        'long waiting time or callback',
                        'request never answered',
                        'there is no answer',
                        'tired of waiting',
                        'website not working']

In [17]:

def agent_cps_df():
    
    agent_categories_name = []

    #Retreive cats name form dictionary

    for values in dict_categories_agent_cps.values():
        if type(values) == list:
            for i in values:
                agent_categories_name.append(i)
        else:
            for i in values.values():
                if type(i)==list:
                    for j in i:
                        agent_categories_name.append(j)
                else: 
                    for h in i.values():
                        for k in h:
                            agent_categories_name.append(k)

    #Info to append
    basic_info=['agentid','eureka id','percent silence','recordingdatetime']

    categories_agent_cps= basic_info + agent_categories_name + TCD_cps


    RawUpdated = read_surveys()
    agent_cps=RawUpdated[categories_agent_cps]

    return agent_cps

def brand_cps_df():

    categories_name = []

    #Retreive cats name form dictionary
    
    for values in dict_categories_brand_cps.values():
        if type(values) == list:
            for i in values:
                categories_name.append(i)
        else:
            for i in values.values():
                if type(i)==list:
                    for j in i:
                        categories_name.append(j)
                else: 
                    for h in i.values():
                        for k in h:
                            categories_name.append(k)

    #Info to append
    basic_info=['agentid','eureka id','percent silence','recordingdatetime']

    categories_brand_cps= basic_info + categories_name + TCD_cps


    RawUpdated = read_surveys()
    brand_cps=RawUpdated[categories_brand_cps]

    return brand_cps

def channel_cps_df():
    #Info to append
    basic_info=['agentid','eureka id','percent silence','recordingdatetime']
    categories_channel_cps= basic_info + channel_categories
    RawUpdated = read_surveys()
    channel_cps=RawUpdated[categories_channel_cps]

    return channel_cps

In [18]:
### AGENT SCORE CPS

def agent_score_cps():

    agent_cps = agent_cps_df()

    # 1.1.1 Opening

    agent_cps["opening_score_cps"]=0

    for cat in dict_categories_agent_cps['opening_cps']['positive']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (10/7) if x==1 else (0)) 
        agent_cps["opening_score_cps"]=agent_cps["opening_score_cps"]+agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['opening_cps']['positive'], axis=1, inplace=True)

    for cat in dict_categories_agent_cps['opening_cps']['negative']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (10/7) if x==0 else (1)) 
        agent_cps["opening_score_cps"]=agent_cps["opening_score_cps"]+agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['opening_cps']['negative'], axis=1, inplace=True)

    # 1.1.2 Investigation

    #agent_cps["investigation_score_cps"]

        #1.1.2.1 Damage from SC

    for cat in dict_categories_agent_cps['investigation_cps']['damage from service center']:
        agent_cps[cat+'_score']=agent_cps['tcd tpc claim status service center'].apply(lambda x: (4) if x==0 else (0))
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 4).where(agent_cps['tcd tpc claim status service center']==1).where(agent_cps[cat]==1, other=agent_cps[cat+'_score'])
        
    agent_cps.drop(dict_categories_agent_cps['investigation_cps']['damage from service center'], axis=1, inplace=True)

        #1.1.2.2 File a claim
        
    agent_cps['file_a_claim_score'] =0


    for cat in dict_categories_agent_cps['investigation_cps']['file a claim']:
        agent_cps[cat+'_score']=agent_cps['tcd tpc file a claim'].apply(lambda x: (2) if x==0 else (0))
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 2).where(agent_cps['tcd tpc file a claim']==1).where(agent_cps[cat]==1, other=agent_cps[cat+'_score'])

        agent_cps['file_a_claim_score'] = agent_cps['file_a_claim_score'] + agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['investigation_cps']['file a claim'], axis=1, inplace=True)


        #1.1.2.3 Food Loss Claim 

    agent_cps['food_loss_claim_score'] =0

    for cat in dict_categories_agent_cps['investigation_cps']['food loss claim']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (2) if x==1 else (0))
        agent_cps['food_loss_claim_score'] =agent_cps['food_loss_claim_score']+agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['investigation_cps']['food loss claim'], axis=1, inplace=True)

        #1.1.2.4 Near Store Question 

    for cat in dict_categories_agent_cps['investigation_cps']['near store question']:   
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (2) if x==1 else (0))


    agent_cps.drop(dict_categories_agent_cps['investigation_cps']['near store question'], axis=1, inplace=True)

        #1.1.2.5 Offer Status

    agent_cps['offer_status_score'] =0 

    for cat in dict_categories_agent_cps['investigation_cps']['offer status']:
        agent_cps[cat+'_score']=agent_cps['tcd tpc claim status  offer status'].apply(lambda x: (0.75) if x==0 else (0))
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 0.75).where(agent_cps['tcd tpc claim status  offer status']==1).where(agent_cps[cat]==1, other=agent_cps[cat+'_score'])

        agent_cps['offer_status_score'] = agent_cps['offer_status_score'] + agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['investigation_cps']['offer status'], axis=1, inplace=True)

        #1.1.2.6 Personalized interaction

    agent_cps['personalized_interaction_score'] =0 

    for cat in dict_categories_agent_cps['investigation_cps']['personalized interaction']:

        if cat == 'agent asks the required information':
            agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (4) if x==1 else (0))
        else:
            agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (2) if x==1 else (0))

        agent_cps['personalized_interaction_score']= agent_cps['personalized_interaction_score'] + agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['investigation_cps']['personalized interaction'], axis=1, inplace=True)


        #1.1.2.7 Registration
        
    agent_cps['registration_score'] =0


    for cat in dict_categories_agent_cps['investigation_cps']['registration']:
        agent_cps[cat+'_score']=agent_cps['tcd tpc registration'].apply(lambda x: (0.8) if x==0 else (0))
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 0.8).where(agent_cps['tcd tpc registration']==1).where(agent_cps[cat]==1, other=agent_cps[cat+'_score'])

        agent_cps['registration_score'] = agent_cps['registration_score'] + agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['investigation_cps']['registration'], axis=1, inplace=True)

        #1.1.2.8 FINAL SCORE INVESTIGATION

    agent_cps['investigation_score'] = (    agent_cps['registration_score'] +
                                            agent_cps['personalized_interaction_score']+
                                            agent_cps['offer_status_score']+
                                            agent_cps['near store_score']+
                                            agent_cps['food_loss_claim_score'] +
                                            agent_cps['file_a_claim_score']+
                                            agent_cps['ask if customer has contacted the sc for damage_score']    )


    # 1.1.3 Resolution

        # 1.1.3.1 Damage from SC 
    

    for cat in dict_categories_agent_cps['resolution_cps']['damage from sc']:
        agent_cps[cat+'_score']=agent_cps['tcd tpc claim status service center'].apply(lambda x: (4) if x==0 else (0))
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 4).where(agent_cps['tcd tpc claim status service center']==1).where(agent_cps[cat]==1, other=agent_cps[cat+'_score'])
        
    agent_cps.drop(dict_categories_agent_cps['resolution_cps']['damage from sc'], axis=1, inplace=True)


        #1.1.3.2 Dead air time 
        
    agent_cps['dead_air_time_score'] =0

    for cat in dict_categories_agent_cps['resolution_cps']['dead air time']['positive']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (2/3) if x==1 else (0))
        agent_cps['dead_air_time_score'] = agent_cps['dead_air_time_score'] + agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['resolution_cps']['dead air time']['positive'], axis=1, inplace=True)

    for cat in dict_categories_agent_cps['resolution_cps']['dead air time']['negative']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (2/3) if x==0 else (1))
        agent_cps['dead_air_time_score'] = agent_cps['dead_air_time_score'] + agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['resolution_cps']['dead air time']['negative'], axis=1, inplace=True)

        #1.1.3.3 Food loss claim

    for cat in dict_categories_agent_cps['resolution_cps']['res_food loss claim']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (3) if x==1 else (0))
            
    agent_cps.drop(dict_categories_agent_cps['resolution_cps']['res_food loss claim'], axis=1, inplace=True)

        #1.1.3.4 SC customer will be contacted

    for cat in dict_categories_agent_cps['resolution_cps']['sc customer will be contacted']:
        agent_cps[cat+'_score']=agent_cps['tcd tpc claim status service center'].apply(lambda x: (4) if x==0 else (0))
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 4).where(agent_cps['tcd tpc claim status service center']==1).where(agent_cps[cat]==1, other=agent_cps[cat+'_score'])
        
    agent_cps.drop(dict_categories_agent_cps['resolution_cps']['sc customer will be contacted'], axis=1, inplace=True)

        #1.1.3.5 Service Center Appointment

    for cat in dict_categories_agent_cps['resolution_cps']['service center appointment']:
        agent_cps[cat+'_score']=agent_cps['tcd tpc claim status service center'].apply(lambda x: (4) if x==0 else (0))
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: 4).where(agent_cps['tcd tpc claim status service center']==1).where(agent_cps[cat]==1, other=agent_cps[cat+'_score'])
        
    agent_cps.drop(dict_categories_agent_cps['resolution_cps']['service center appointment'], axis=1, inplace=True)

        ## 1.1.3.6 RESULTION CPS FINAL SCORE

    agent_cps['resolution_cps_score']=( agent_cps['you need to contact the sc_score']+
                                        agent_cps['dead_air_time_score']+
                                        agent_cps['recite script_score']+
                                        agent_cps['customer will be contacted_score']+
                                        agent_cps['status of the appointment or missed_score'])

    # 1.1.4 Closing

    agent_cps['closing_cps_score']=0

        #1.1.4.1 Closing call 

    for cat in dict_categories_agent_cps['closing_cps'][ 'closing call cps']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (2) if x==1 else (0))
        agent_cps['closing_cps_score']=agent_cps['closing_cps_score']+agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['closing_cps'][ 'closing call cps'], axis=1, inplace=True)

        #1.1.4.2 Transfering cat CPS
    for cat in dict_categories_agent_cps['closing_cps'][ 'transfering cat cps']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (1) if x==1 else (0))
        agent_cps['closing_cps_score']=agent_cps['closing_cps_score']+agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['closing_cps'][ 'transfering cat cps'], axis=1, inplace=True)


    # 1.1.5 Legal

    agent_cps['legal_cps_score']=0

        #1.1.5.1 Legal Responsibility 

    for cat in dict_categories_agent_cps['legal_cps']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (20/3) if x==0 else (1))
        agent_cps['legal_cps_score']=agent_cps['legal_cps_score']+agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['legal_cps'], axis=1, inplace=True)


    # 1.1.6 Amynta Experiences CPS

    agent_cps['amynta_experiences_cps_score']=0

        #1.1.6.1 Amynta Experiences

    #Negatives
    for cat in dict_categories_agent_cps['amynta_experiences']['negative']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (2.5) if x==0 else (0))
        agent_cps['amynta_experiences_cps_score']=agent_cps['amynta_experiences_cps_score']+agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['amynta_experiences']['negative'], axis=1, inplace=True)

    #Positives
    for cat in dict_categories_agent_cps['amynta_experiences']['positive']:
        agent_cps[cat+'_score']=agent_cps[cat].apply(lambda x: (2.5) if x==1 else (0))
        agent_cps['amynta_experiences_cps_score']=agent_cps['amynta_experiences_cps_score']+agent_cps[cat+'_score']

    agent_cps.drop(dict_categories_agent_cps['amynta_experiences']['positive'], axis=1, inplace=True)

    #1.1.7  AGENT CPS TOTAL SCORE ######

    agent_cps['agent_total_score'] = (   agent_cps['amynta_experiences_cps_score']+
                                                    agent_cps['legal_cps_score']+
                                                    agent_cps['closing_cps_score']+
                                                    agent_cps['resolution_cps_score']+
                                                    agent_cps['investigation_score']+
                                                    agent_cps["opening_score_cps"]          )

    ## Drop TCD
    agent_cps.drop(TCD_cps, axis=1, inplace=True)

    ### EXPORT TABLE AGENT CPS 


    agent_cps.to_excel('output/agent_cps_scores.xlsx', index=False)

    

## BRAND SCORE CPS
def agent_score_cps():
    brand_cps=brand_cps_df()

    brand_cps["branding_score_cps"]=0

    brand_cps['contract term and conditions_score']=0
    for cat in dict_categories_brand_cps['contract term and conditions']:
        brand_cps[cat+'_score']=brand_cps[cat].apply(lambda x: (38/3) if x==1 else (0))
        brand_cps["branding_score_cps"]=brand_cps["branding_score_cps"]+brand_cps[cat+'_score']
        brand_cps['contract term and conditions_score']=brand_cps['contract term and conditions_score']+brand_cps[cat+'_score']
    #brand_cps.drop(dict_categories_brand_cps['contract term and conditions'], axis=1, inplace=True)


    brand_cps['service center issues_score']=0
    for cat in dict_categories_brand_cps['service center issues']:
        brand_cps[cat+'_score']=brand_cps['tcd tpc claim status service center'].apply(lambda x: (8) if x==0 else (0))
        brand_cps[cat+'_score']=brand_cps[cat].apply(lambda x: (8)).where(brand_cps['tcd tpc claim status service center']==1).where(brand_cps[cat]==1, other=brand_cps[cat+'_score'])
        brand_cps["branding_score_cps"]=brand_cps["branding_score_cps"]+brand_cps[cat+'_score']
        brand_cps['service center issues_score']=brand_cps['service center issues_score']+brand_cps[cat+'_score']
    #brand_cps.drop(dict_categories_brand_cps['service center issues'], axis=1, inplace=True)
    #agent_cps.drop(TCD_cps, axis=1, inplace=True)

    brand_cps.to_excel('output/brand_cps_scores.xlsx', index=False)


#1.3 Channel 
def agent_score_cps():

        channel_cps = channel_cps_df()
        channel_cps["channel_score_cps"]=0 

        for cat in channel_categories:
                channel_cps[cat+'_score']=channel_cps[cat].apply(lambda x: (100/6) if x==0 else (0))
                channel_cps["channel_score_cps"]=channel_cps["channel_score_cps"]+channel_cps[cat+'_score']
                
        channel_cps.to_excel('output/channel_cps_scores.xlsx', index=False)


In [56]:
dict_categories_agent_gman= {

    'opening_gman' : {

        'customer engagement':{

            'negative':[    'listening skills gman',
                            'no acknowledge gman',
                            'no urgency to assist gman',
                            'professional language gman'], 

            'positive' : ['willingness to assist gman'],
                                },
        'proper greeting gman' : ['opening gman']
                        },

    'investigation gman': {
        'personalized interactions': [  'agent asks the required information gman',
                                        'callers name gman' ],
        'registration gman' :       ['account holder name',
                                    'date of purchase delivery',
                                    'email',
                                    'home address',
                                    'phone number',
                                    'relation with account holder',
                                    'service agreement number',
                                    'store address',
                                    'store|retailer of purchase',
                                    'what type of plan']
                            },

    'resolution gman' : ['agent did not met dead air time gman',
                        'agent did not met hold time gman'],

    'closing gman' : {
        'closing call gman' : ['closing gman'],

        'closing gman transfers cat' : ['transfer to supervisor gman',
                                        'transfers gman']
                    },

    'legal gman' : [    'confidential information gman',
                        'derogatory manner gman',
                        'internal procedures gman'],
    
    'amynta experiences gman' : ['blames the customer gman',
                                'did not communicates with respect gman',
                                'empathy to the customer situation gman',
                                'politics and religions gman']

    }


dict_categories_brand_gman= [   'delays on claim process gman',
                                'no timely response from sc gman',
                                'the customer cant reach the sc gman',
                                'the customer perceives unsatisfactory service gman']

dict_categories_channel_gman={
    
    '17':       [   'application is not running gman',
                    'delay when transfering the call gman',
                    'not good communication gman',
                    'website not working gman'],
    '10.66':    [   'been on hold for so long gman',
                    'no response from other departments gman',
                    'request never answered gman']}

In [38]:
def df_gman():

    dicts = [dict_categories_channel_gman,dict_categories_agent_gman,dict_categories_brand_gman]
    RawUpdated = read_surveys()

    for subject in dicts:
        cat_names = []

        if type(subject) == dict:
            for values in subject.values():
                if type(values) == list:
                    for i in values:
                        cat_names.append(i)
                else:
                    for i in values.values():
                        if type(i)==list:
                            for j in i:
                                cat_names.append(j)
                        else: 
                            for h in i.values():
                                for k in h:
                                    cat_names.append(k)
        else:
            cat_names = subject
    #Info to append
    
        basic_info=['agentid','eureka id']

        cat_names= basic_info + cat_names + TCD_cps

        if subject is dict_categories_channel_gman:
            channel_gman=RawUpdated[cat_names]
        if subject is dict_categories_agent_gman:
            agent_gman = RawUpdated[cat_names]
        if subject is dict_categories_brand_gman:
            brand_gman = RawUpdated[cat_names]
        

    return agent_gman, channel_gman, brand_gman
    

In [59]:
agent_gman, channel_gman, brand_gman = df_gman()

###########################################     SCORES GMAN
#  
## AGENT GMAN 

# 2.1.1 Opening

agent_gman["opening_total_score_gman"]=0

for cat in dict_categories_agent_gman['opening_gman']['customer engagement']['positive']:
    agent_gman[cat+'_score']=agent_gman[cat].apply(lambda x: (3.2) if x==1 else (0)) 
    agent_gman["opening_total_score_gman"]=agent_gman["opening_total_score_gman"]+agent_gman[cat+'_score']

agent_gman.drop(dict_categories_agent_gman['opening_gman']['customer engagement']['positive'], axis=1, inplace=True)

for cat in dict_categories_agent_gman['opening_gman']['customer engagement']['negative']:
    agent_gman[cat+'_score']=agent_gman[cat].apply(lambda x: (3.2) if x==0 else (1)) 
    agent_gman["opening_total_score_gman"]=agent_gman["opening_total_score_gman"]+agent_gman[cat+'_score']

agent_gman.drop(dict_categories_agent_gman['opening_gman']['customer engagement']['negative'], axis=1, inplace=True)

for cat in dict_categories_agent_gman['opening_gman']['proper greeting gman']:
    agent_gman[cat+'_score']=agent_gman[cat].apply(lambda x: (4) if x==1 else (0)) 
    agent_gman["opening_total_score_gman"]=agent_gman["opening_total_score_gman"]+agent_gman[cat+'_score']

agent_gman.drop(dict_categories_agent_gman['opening_gman']['proper greeting gman'], axis=1, inplace=True)


# 2.1.2 Investigation

agent_gman['investigation_total_score'] = 0 


    #2.1.2.1 Perzonalized interactions
agent_gman['personalized interactions_score'] = 0 



for cat in dict_categories_agent_gman['investigation gman']['personalized interactions']:

    if cat == 'agent asks the required information gman':
        agent_gman[cat+'_score']=agent_gman[cat].apply(lambda x: (11) if x==1 else (0)) 

    if cat == 'callers name gman':
        agent_gman[cat+'_score']=agent_gman[cat].apply(lambda x: (2) if x==1 else (0))

    
    agent_gman['investigation_total_score'] = agent_gman['investigation_total_score'] + agent_gman[cat+'_score']
    agent_gman['personalized interactions_score'] = agent_gman['personalized interactions_score'] + agent_gman[cat+'_score']
    
agent_gman.drop(dict_categories_agent_gman['investigation gman']['personalized interactions'], axis=1, inplace=True)


    #2.1.2.2 Registration 

agent_gman['registration_score'] = 0 

for cat in dict_categories_agent_gman['investigation gman']['registration gman']:

    agent_gman[cat+'_score']=agent_gman['tcd tpc registration'].apply(lambda x: (1.8) if x==0 else (0))
    agent_gman[cat+'_score']=agent_gman[cat].apply(lambda x: 1.8).where(agent_gman['tcd tpc registration']==1).where(agent_gman[cat]==1, other=agent_gman[cat+'_score'])

    agent_gman['registration_score'] = agent_gman['registration_score'] + agent_gman[cat+'_score']
    agent_gman['investigation_total_score'] = agent_gman['investigation_total_score'] + agent_gman[cat+'_score']


agent_gman.drop(dict_categories_agent_gman['investigation gman']['registration gman'], axis=1, inplace=True)

# 2.1.3 Resolution

agent_gman['resolution_total_score'] = 0 


for cat in dict_categories_agent_gman['resolution gman']:
    agent_gman[cat+'_score']=agent_gman[cat].apply(lambda x: (2) if x==0 else (0)) 
    agent_gman['resolution_total_score']=agent_gman['resolution_total_score']+agent_gman[cat+'_score']
    
#agent_gman.drop(dict_categories_agent_gman['resolution gman'], axis=1, inplace=True)


#2.1.4 Closing 

for cat in dict_categories_agent_gman['closing gman']['closing call gman']:
    agent_gman[cat+'_total_score']=agent_gman[cat].apply(lambda x: (11) if x==0 else (0)) 
    




agent_gman.to_excel('test.xlsx')